## Train

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd
from datetime import datetime
from pathlib import Path
import numpy as np

def create_model(num_classes):
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Enhanced feature extraction layers
    x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    predictions = Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Initially freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    return model, base_model

def train(train_csv_path, train_images_dir, output_dir):
    train_csv_path = Path(train_csv_path)
    train_images_dir = Path(train_images_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Loading training data from: {train_csv_path}")
    train_df = pd.read_csv(train_csv_path)
    constellation_cols = [col for col in train_df.columns if col != 'filename']
    
    # Enhanced data augmentation
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        zoom_range=0.3,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.7, 1.3],
        fill_mode='nearest',
        validation_split=0.2,
        shear_range=0.2
    )
    
    batch_size = 16
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=str(train_images_dir),
        x_col='filename',
        y_col=constellation_cols,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='raw',
        subset='training'
    )
    
    validation_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=str(train_images_dir),
        x_col='filename',
        y_col=constellation_cols,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='raw',
        subset='validation'
    )
    
    print("Creating model with DenseNet201 architecture...")
    model, base_model = create_model(len(constellation_cols))
    
    # Custom learning rate schedule
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9
    learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps, decay_rate
    )
    
    # Modified metrics for multi-label classification
    model.compile(
        optimizer=Adam(learning_rate=learning_rate_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy', 
                tf.keras.metrics.AUC(multi_label=True),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall()]
    )
    
    # Callbacks
    model_path = output_dir / 'constellation_model_densenet.keras'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        str(model_path),
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
    
    log_dir = output_dir / "logs" / f"densenet_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=str(log_dir),
        histogram_freq=1
    )
    
    # Phase 1: Training top layers
    print("Phase 1: Training top layers...")
    history1 = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=20,
        callbacks=[checkpoint, early_stopping, reduce_lr, tensorboard_callback]
    )
    
    print("Phase 2: Fine-tuning...")
    # Unfreeze last 30% of the base model layers
    num_layers_to_unfreeze = int(len(base_model.layers) * 0.3)
    for layer in model.layers[-num_layers_to_unfreeze:]:
        layer.trainable = True

    # Create a new optimizer with a lower learning rate schedule
    new_learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_learning_rate / 10,
        decay_steps=decay_steps,
        decay_rate=decay_rate
    )
    new_optimizer = Adam(learning_rate=new_learning_rate_schedule)

    # Recompile the model with the new optimizer
    model.compile(
        optimizer=new_optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy',
             tf.keras.metrics.AUC(multi_label=True),
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()]
    )

    # Train the model
    history2 = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=20,
        callbacks=[checkpoint, early_stopping, reduce_lr, tensorboard_callback]
    )

    
    # Save labels
    labels_path = output_dir / 'constellation_labels.txt'
    with open(labels_path, 'w') as f:
        f.write('\n'.join(constellation_cols))
    
    print(f"Training completed. Model saved to: {model_path}")
    print(f"Labels saved to: {labels_path}")
    print(f"TensorBoard logs saved to: {log_dir}")
    
    return history1, history2

if __name__ == '__main__':
    train_csv_path = r"D:\programs\astro_web\pretrained_model\train\_classes.csv"
    train_images_dir = r"D:\programs\astro_web\pretrained_model\train"
    output_dir = r"D:\programs\astro_web\pretrained_model\output"
    
    try:
        history1, history2 = train(train_csv_path, train_images_dir, output_dir)
    except Exception as e:
        print(f"Training failed: {e}")

Loading training data from: D:\programs\astro_web\pretrained_model\train\_classes.csv
Found 1313 validated image filenames.
Found 328 validated image filenames.
Creating model with DenseNet201 architecture...
Phase 1: Training top layers...


C:\Users\soure\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


KeyboardInterrupt: 

## Validation

In [20]:
pip cache purge

Files removed: 116
Note: you may need to restart the kernel to use updated packages.


In [32]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime

def load_model_and_labels(model_path, labels_path):
    """
    Load the trained model and corresponding class labels.
    
    Args:
        model_path: Path to the saved Keras model
        labels_path: Path to the text file containing class labels
    
    Returns:
        model: Loaded Keras model
        labels: List of class labels
    """
    model = tf.keras.models.load_model(model_path)
    with open(labels_path, 'r') as f:
        labels = f.read().splitlines()
    return model, labels

def create_validation_generator(val_csv_path, val_images_dir, labels, batch_size=16):
    """
    Create a data generator for validation data.
    
    Args:
        val_csv_path: Path to validation CSV file
        val_images_dir: Directory containing validation images
        labels: List of class labels
        batch_size: Batch size for validation
    
    Returns:
        validation_generator: DataFrameIterator for validation data
        val_df: Validation DataFrame
    """
    val_df = pd.read_csv(val_csv_path)
    
    # Ensure all label columns exist in the validation DataFrame
    for label in labels:
        if label not in val_df.columns:
            raise ValueError(f"Label '{label}' not found in validation data")
    
    validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255  # Only rescaling for validation, no augmentation
    )
    
    validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=str(val_images_dir),
        x_col='filename',
        y_col=labels,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='raw',
        shuffle=False  # Important for maintaining order in predictions
    )
    
    return validation_generator, val_df

def calculate_optimal_thresholds(model, validation_generator, labels):
    """
    Calculate optimal prediction thresholds for each class using validation data.
    
    Args:
        model: Trained Keras model
        validation_generator: DataFrameIterator for validation data
        labels: List of class labels
    
    Returns:
        thresholds: Dictionary of optimal thresholds for each class
    """
    predictions = model.predict(validation_generator)
    true_labels = validation_generator.labels
    
    thresholds = {}
    threshold_metrics = {}
    
    for i, label in enumerate(labels):
        best_f1 = 0
        best_threshold = 0.5
        best_metrics = None
        
        # Try different thresholds
        for threshold in np.arange(0.3, 0.7, 0.05):
            pred_binary = (predictions[:, i] > threshold).astype(int)
            true_binary = true_labels[:, i]
            
            # Calculate metrics
            tp = np.sum((pred_binary == 1) & (true_binary == 1))
            fp = np.sum((pred_binary == 1) & (true_binary == 0))
            fn = np.sum((pred_binary == 0) & (true_binary == 1))
            tn = np.sum((pred_binary == 0) & (true_binary == 0))
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
                best_metrics = {
                    'threshold': threshold,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'tp': tp,
                    'fp': fp,
                    'fn': fn,
                    'tn': tn
                }
        
        thresholds[label] = best_threshold
        threshold_metrics[label] = best_metrics
    
    return thresholds, threshold_metrics

def generate_metrics_report(predictions, true_labels, labels, thresholds, output_dir):
    """
    Generate and save detailed metrics report.
    
    Args:
        predictions: Model predictions
        true_labels: True labels
        labels: List of class labels
        thresholds: Dictionary of thresholds for each class
        output_dir: Directory to save reports
    """
    # Create metrics directory
    metrics_dir = Path(output_dir) / 'validation_metrics'
    metrics_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate classification report for each class
    report_data = []
    confusion_matrices = {}
    
    for i, label in enumerate(labels):
        pred_binary = (predictions[:, i] > thresholds[label]).astype(int)
        true_binary = true_labels[:, i]
        
        # Generate confusion matrix
        cm = confusion_matrix(true_binary, pred_binary)
        confusion_matrices[label] = cm
        
        # Calculate metrics
        report = classification_report(true_binary, pred_binary, output_dict=True)
        report_data.append({
            'Label': label,
            'Precision': report['1']['precision'],
            'Recall': report['1']['recall'],
            'F1-Score': report['1']['f1-score'],
            'Support': report['1']['support'],
            'Threshold': thresholds[label]
        })
    
    # Save metrics report
    metrics_df = pd.DataFrame(report_data)
    metrics_df.to_csv(metrics_dir / 'validation_metrics.csv', index=False)
    
    # Save confusion matrices in a readable format
    with open(metrics_dir / 'confusion_matrices.txt', 'w') as f:
        for label, cm in confusion_matrices.items():
            f.write(f"\nConfusion Matrix for {label}:\n")
            f.write("True Negative  False Positive\n")
            f.write("False Negative True Positive\n")
            f.write(str(cm))
            f.write("\n" + "="*40 + "\n")
    
    return metrics_df, confusion_matrices

def validate(model_path, val_csv_path, val_images_dir, output_dir):
    """
    Main validation function.
    
    Args:
        model_path: Path to the saved model
        val_csv_path: Path to validation CSV file
        val_images_dir: Directory containing validation images
        output_dir: Directory to save validation results
    """
    # Convert paths to Path objects
    model_path = Path(model_path)
    val_csv_path = Path(val_csv_path)
    val_images_dir = Path(val_images_dir)
    output_dir = Path(output_dir)
    
    # Load model and labels
    labels_path = model_path.parent / 'constellation_labels.txt'
    print("Loading model and labels...")
    model, labels = load_model_and_labels(model_path, labels_path)
    
    # Create validation generator
    print("Creating validation generator...")
    validation_generator, val_df = create_validation_generator(val_csv_path, val_images_dir, labels)
    
    # Calculate optimal thresholds
    print("Calculating optimal thresholds...")
    thresholds, threshold_metrics = calculate_optimal_thresholds(model, validation_generator, labels)
    
    # Generate predictions
    print("Generating predictions...")
    predictions = model.predict(validation_generator)
    
    # Generate and save metrics report
    print("Generating metrics report...")
    metrics_df, confusion_matrices = generate_metrics_report(
        predictions,
        validation_generator.labels,
        labels,
        thresholds,
        output_dir
    )
    
    # Print summary results
    print("\nValidation Results Summary:")
    print(metrics_df.to_string(index=False))
    
    print("\nConfusion Matrices:")
    for label, cm in confusion_matrices.items():
        print(f"\n{label}:")
        print("Format: [[TN, FP], [FN, TP]]")
        print(cm)
    
    print(f"\nDetailed metrics saved to: {output_dir}/validation_metrics/")

if __name__ == '__main__':
    model_path = r"D:\programs\astro_web\pretrained_model\output\constellation_model_densenet.keras"
    val_csv_path = r"D:\programs\astro_web\pretrained_model\valid\_classes.csv"
    val_images_dir = r"D:\programs\astro_web\pretrained_model\valid"
    output_dir = r"D:\programs\astro_web\pretrained_model\output"
    
    try:
        validate(model_path, val_csv_path, val_images_dir, output_dir)
    except Exception as e:
        print(f"Validation failed: {e}")

Loading model and labels...
Creating validation generator...
Found 469 validated image filenames.
Calculating optimal thresholds...


C:\Users\soure\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step
Generating predictions...
30/30 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step
Generating metrics report...

Validation Results Summary:
       Label  Precision   Recall  F1-Score  Support  Threshold
      aquila   0.969697 0.941176  0.955224     68.0       0.30
      bootes   0.872093 0.986842  0.925926     76.0       0.30
 canis_major   1.000000 0.913043  0.954545     69.0       0.30
 canis_minor   0.965116 0.976471  0.970760     85.0       0.30
  cassiopeia   0.862069 0.986842  0.920245    152.0       0.35
      cygnus   0.862745 0.888889  0.875622     99.0       0.35
      gemini   1.000000 0.934783  0.966292     92.0       0.40
         leo   0.939394 0.885714  0.911765     70.0       0.65
        lyra   0.990476 0.920354  0.954128    113.0       0.30
        moon   1.000000 0.846847  0.917073    111.0       0.30
       orion   0.960000 0.969697  0.964824     99.0       0.55
    pleiades   0.866197 0.924812  0.894545    133.0       0.55
 sagittarius   1.00

## Testing

In [34]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime

def load_model_and_metadata(model_path):
    """
    Load the trained model, labels, and optimal thresholds from validation.
    
    Args:
        model_path: Path to the saved model
        
    Returns:
        model: Loaded Keras model
        labels: List of class labels
        thresholds: Dictionary of optimal thresholds for each class
    """
    # Load the model
    model = tf.keras.models.load_model(model_path)
    
    # Load the labels
    labels_path = Path(model_path).parent / 'constellation_labels.txt'
    with open(labels_path, 'r') as f:
        labels = f.read().splitlines()
    
    # Load optimal thresholds from validation
    metrics_path = Path(model_path).parent / 'validation_metrics' / 'validation_metrics.csv'
    if metrics_path.exists():
        metrics_df = pd.read_csv(metrics_path)
        thresholds = dict(zip(metrics_df['Label'], metrics_df['Threshold']))
    else:
        print("Warning: Validation metrics not found. Using default threshold of 0.5")
        thresholds = {label: 0.5 for label in labels}
    
    return model, labels, thresholds

def create_test_generator(test_csv_path, test_images_dir, labels, batch_size=16):
    """
    Create a data generator for test data.
    
    Args:
        test_csv_path: Path to test CSV file
        test_images_dir: Directory containing test images
        labels: List of class labels
        batch_size: Batch size for testing
        
    Returns:
        test_generator: DataFrameIterator for test data
        test_df: Test DataFrame
        filenames: List of image filenames
    """
    test_df = pd.read_csv(test_csv_path)
    
    # Verify all required columns exist
    for label in labels:
        if label not in test_df.columns:
            raise ValueError(f"Label '{label}' not found in test data")
    
    # Create test data generator with only rescaling
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=str(test_images_dir),
        x_col='filename',
        y_col=labels,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='raw',
        shuffle=False
    )
    
    return test_generator, test_df, list(test_df['filename'])

def evaluate_predictions(predictions, true_labels, labels, thresholds):
    """
    Evaluate model predictions using various metrics.
    
    Args:
        predictions: Model predictions
        true_labels: True labels
        labels: List of class labels
        thresholds: Dictionary of thresholds for each class
        
    Returns:
        metrics_df: DataFrame containing evaluation metrics
        confusion_matrices: Dictionary of confusion matrices for each class
    """
    evaluation_results = []
    confusion_matrices = {}
    
    for i, label in enumerate(labels):
        # Apply threshold to get binary predictions
        pred_binary = (predictions[:, i] > thresholds[label]).astype(int)
        true_binary = true_labels[:, i]
        
        # Calculate confusion matrix
        cm = confusion_matrix(true_binary, pred_binary)
        confusion_matrices[label] = cm
        
        # Calculate detailed metrics
        report = classification_report(true_binary, pred_binary, output_dict=True)
        
        # Calculate additional metrics
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value
        
        evaluation_results.append({
            'Label': label,
            'Precision': report['1']['precision'],
            'Recall': report['1']['recall'],
            'F1-Score': report['1']['f1-score'],
            'Specificity': specificity,
            'NPV': npv,
            'Support': report['1']['support'],
            'Threshold': thresholds[label]
        })
    
    return pd.DataFrame(evaluation_results), confusion_matrices

def save_test_results(metrics_df, confusion_matrices, predictions, filenames, labels, output_dir):
    """
    Save all test results to files.
    
    Args:
        metrics_df: DataFrame containing evaluation metrics
        confusion_matrices: Dictionary of confusion matrices
        predictions: Raw model predictions
        filenames: List of image filenames
        labels: List of class labels
        output_dir: Directory to save results
    """
    # Create test results directory
    results_dir = Path(output_dir) / 'test_results'
    results_dir.mkdir(parents=True, exist_ok=True)
    
    # Save metrics to CSV
    metrics_df.to_csv(results_dir / 'test_metrics.csv', index=False)
    
    # Save confusion matrices
    with open(results_dir / 'test_confusion_matrices.txt', 'w') as f:
        f.write("Confusion Matrix Format:\n")
        f.write("[[TN, FP],\n [FN, TP]]\n\n")
        for label, cm in confusion_matrices.items():
            f.write(f"\n{label}:\n")
            f.write(str(cm))
            f.write("\n" + "="*40 + "\n")
    
    # Save detailed predictions
    predictions_df = pd.DataFrame(predictions, columns=labels)
    predictions_df['filename'] = filenames
    predictions_df.to_csv(results_dir / 'raw_predictions.csv', index=False)

def test_model(model_path, test_csv_path, test_images_dir, output_dir):
    """
    Main testing function.
    
    Args:
        model_path: Path to the saved model
        test_csv_path: Path to test CSV file
        test_images_dir: Directory containing test images
        output_dir: Directory to save test results
    """
    print("Starting model testing...")
    
    # Load model and metadata
    print("Loading model and metadata...")
    model, labels, thresholds = load_model_and_metadata(model_path)
    
    # Create test generator
    print("Creating test data generator...")
    test_generator, test_df, filenames = create_test_generator(test_csv_path, test_images_dir, labels)
    
    # Generate predictions
    print("Generating predictions on test data...")
    predictions = model.predict(test_generator)
    
    # Evaluate predictions
    print("Evaluating predictions...")
    metrics_df, confusion_matrices = evaluate_predictions(
        predictions,
        test_generator.labels,
        labels,
        thresholds
    )
    
    # Save results
    print("Saving test results...")
    save_test_results(metrics_df, confusion_matrices, predictions, filenames, labels, output_dir)
    
    # Print summary results
    print("\nTest Results Summary:")
    print(metrics_df.to_string(index=False))
    
    print("\nConfusion Matrices:")
    for label, cm in confusion_matrices.items():
        print(f"\n{label}:")
        tn, fp, fn, tp = cm.ravel()
        print(f"True Negatives: {tn}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"True Positives: {tp}")
    
    print(f"\nDetailed results saved to: {output_dir}/test_results/")

if __name__ == '__main__':
    model_path = r"D:\programs\astro_web\pretrained_model\output\constellation_model_densenet.keras"
    test_csv_path = r"D:\programs\astro_web\pretrained_model\test\_classes.csv"
    test_images_dir = r"D:\programs\astro_web\pretrained_model\test"
    output_dir = r"D:\programs\astro_web\pretrained_model\output"
    
    try:
        test_model(model_path, test_csv_path, test_images_dir, output_dir)
    except Exception as e:
        print(f"Testing failed: {e}")

Starting model testing...
Loading model and metadata...
Creating test data generator...
Found 234 validated image filenames.
Generating predictions on test data...


C:\Users\soure\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step
Evaluating predictions...
Saving test results...

Test Results Summary:
       Label  Precision   Recall  F1-Score  Specificity      NPV  Support  Threshold
      aquila   1.000000 1.000000  1.000000     1.000000 1.000000     31.0       0.30
      bootes   0.868421 0.970588  0.916667     0.975000 0.994898     34.0       0.30
 canis_major   1.000000 0.909091  0.952381     1.000000 0.985294     33.0       0.30
 canis_minor   0.925000 1.000000  0.961039     0.984772 1.000000     37.0       0.30
  cassiopeia   0.843750 1.000000  0.915254     0.901961 1.000000     81.0       0.35
      cygnus   0.854545 0.870370  0.862385     0.955556 0.960894     54.0       0.35
      gemini   0.974359 0.950000  0.962025     0.994845 0.989744     40.0       0.40
         leo   0.888889 0.888889  0.888889     0.979798 0.979798     36.0       0.65
        lyra   0.909091 0.961538  0.934579     0.972527 0.988827     52.0       0.30
        moon   1.000000 0.833333  0.909

In [35]:
pip install tensorflow opencv-python pillow numpy

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
    --------------------------------------- 0.5/38.8 MB 2.4 MB/s eta 0:00:17
   - -------------------------------------- 1.0/38.8 MB 2.0 MB/s eta 0:00:19
   - -------------------------------------- 1.3/38.8 MB 2.2 MB/s eta 0:00:18
   - -------------------------------------- 1.8/38.8 MB 2.1 MB/s eta 0:00:18
   -- ------------------------------------- 2.4/38.8 MB 2.2 MB/s eta 0:00:17
   -- ------------------------------------- 2.9/38.8 MB 2.2 MB/s eta 0:00:17
   --- ------------------------------------ 3.4/38.8 MB 2.2 MB/s eta 0:00:17
   ---- ----------------------------------- 3.9/38.8 MB 2.2 MB/s eta 0:00:16
   ---- ----------------------------------- 4.5/38.8 MB 2.3 MB/s eta 0:00:15
   ----- ---------------------------------- 5.0/38.8 MB 2.3 MB/s eta 0:00:15
   ----- ---------------------------------- 5.2/38.8 MB 2.3 MB/s eta 0:00:15
   ----- ----

In [1]:
import tensorflow as tf
import numpy as np
import cv2
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

class ConstellationDetector:
    def __init__(self, model_path):
        """
        Initialize the constellation detector with the trained model.
        
        Args:
            model_path: Path to the trained model file
        """
        # Load the model and labels
        self.model = tf.keras.models.load_model(model_path)
        
        # Load labels and thresholds
        model_dir = Path(model_path).parent
        with open(model_dir / 'constellation_labels.txt', 'r') as f:
            self.labels = f.read().splitlines()
            
        # Load thresholds if available, otherwise use default
        metrics_path = model_dir / 'validation_metrics' / 'validation_metrics.csv'
        if metrics_path.exists():
            import pandas as pd
            metrics_df = pd.read_csv(metrics_path)
            self.thresholds = dict(zip(metrics_df['Label'], metrics_df['Threshold']))
        else:
            self.thresholds = {label: 0.5 for label in self.labels}
        
        # Initialize the GUI
        self.setup_gui()
    
    def setup_gui(self):
        """Set up the graphical user interface."""
        self.root = tk.Tk()
        self.root.title("Constellation Detector")
        self.root.geometry("1200x800")
        
        # Create main frame
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Create buttons frame
        button_frame = tk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Add buttons
        tk.Button(button_frame, text="Load Image", command=self.load_image).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="Detect Constellations", command=self.process_image).pack(side=tk.LEFT, padx=5)
        
        # Create image frame
        self.image_frame = tk.Frame(main_frame)
        self.image_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create results frame
        self.results_frame = tk.Frame(main_frame)
        self.results_frame.pack(fill=tk.X, pady=(10, 0))
        
        self.image_label = tk.Label(self.image_frame)
        self.image_label.pack()
        
        self.results_text = tk.Text(self.results_frame, height=5)
        self.results_text.pack(fill=tk.X)
        
        self.current_image = None
        self.current_image_path = None
    
    def load_image(self):
        """Open a file dialog to select an image."""
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.gif *.tiff")]
        )
        if file_path:
            self.current_image_path = file_path
            self.display_image(file_path)
    
    def display_image(self, image_path, highlighted_image=None):
        """
        Display the image in the GUI.
        
        Args:
            image_path: Path to the image file
            highlighted_image: Optional numpy array of highlighted image
        """
        if highlighted_image is not None:
            # Convert BGR to RGB for PIL
            image = cv2.cvtColor(highlighted_image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)
        else:
            image = Image.open(image_path)
        
        # Resize image to fit the window while maintaining aspect ratio
        display_size = (800, 600)
        image.thumbnail(display_size, Image.ANTIALIAS)

        
        # Convert to PhotoImage for tkinter
        photo = ImageTk.PhotoImage(image)
        self.image_label.configure(image=photo)
        self.image_label.image = photo  # Keep a reference
    
    def preprocess_image(self, image_path):
        """
        Preprocess the image for model input.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            processed_image: Preprocessed image ready for model input
            original_image: Original image for visualization
        """
        # Read and resize image
        image = cv2.imread(image_path)
        original_image = image.copy()
        
        # Resize to model input size
        processed_image = cv2.resize(image, (224, 224))
        processed_image = processed_image.astype('float32') / 255.0
        
        return np.expand_dims(processed_image, axis=0), original_image
    
    def process_image(self):
        """Process the loaded image and display results."""
        if self.current_image_path is None:
            messagebox.showerror("Error", "Please load an image first")
            return
        
        # Preprocess image
        input_tensor, original_image = self.preprocess_image(self.current_image_path)
        
        # Get predictions
        predictions = self.model.predict(input_tensor)
        
        # Get detected constellations
        detected_constellations = []
        for i, label in enumerate(self.labels):
            if predictions[0][i] > self.thresholds[label]:
                confidence = predictions[0][i] * 100
                detected_constellations.append((label, confidence))
        
        # Highlight detected constellations
        highlighted_image = self.highlight_constellations(original_image, detected_constellations)
        
        # Update display
        self.display_image(self.current_image_path, highlighted_image)
        
        # Update results text
        self.results_text.delete(1.0, tk.END)
        if detected_constellations:
            results = "Detected Constellations:\n"
            for const, conf in detected_constellations:
                results += f"{const}: {conf:.1f}% confidence\n"
        else:
            results = "No constellations detected in the image."
        self.results_text.insert(tk.END, results)
    
    def highlight_constellations(self, image, detected_constellations):
        """
        Add visual highlights and labels for detected constellations.
        
        Args:
            image: Original image
            detected_constellations: List of (constellation_name, confidence) tuples
            
        Returns:
            highlighted_image: Image with highlights and labels
        """
        highlighted_image = image.copy()
        
        # Add semi-transparent overlay
        overlay = highlighted_image.copy()
        
        # Add text with detected constellations
        height = highlighted_image.shape[0]
        for i, (const, conf) in enumerate(detected_constellations):
            # Position text at the top of the image
            y_position = 30 + (i * 30)
            cv2.putText(
                highlighted_image,
                f"{const} ({conf:.1f}%)",
                (10, y_position),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 0),
                2
            )
        
        # Add subtle highlighting effect
        cv2.addWeighted(overlay, 0.2, highlighted_image, 0.8, 0, highlighted_image)
        
        return highlighted_image
    
    def run(self):
        """Start the application."""
        self.root.mainloop()

def main():
    # Set up paths
    model_path = r"D:\programs\astro_web\pretrained_model\output\constellation_model_densenet.keras"
    
    try:
        # Create and run the detector
        detector = ConstellationDetector(model_path)
        detector.run()
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\soure\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\soure\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\soure\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\soure\anaconda3\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "C:\Users\soure\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\soure\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\soure\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\soure\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\soure\anaconda3\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "C:\Users\soure\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


In [2]:
import tensorflow as tf
import numpy as np
import cv2
from pathlib import Path
from tkinter import filedialog, Tk
import os

class ConstellationDetector:
    def __init__(self, model_path):
        """
        Initialize the constellation detector with the trained model.
        
        Args:
            model_path: Path to the trained model file
        """
        # Load the model and labels
        self.model = tf.keras.models.load_model(model_path)
        
        # Load labels and thresholds
        model_dir = Path(model_path).parent
        with open(model_dir / 'constellation_labels.txt', 'r') as f:
            self.labels = f.read().splitlines()
            
        # Load thresholds if available, otherwise use default
        metrics_path = model_dir / 'validation_metrics' / 'validation_metrics.csv'
        if metrics_path.exists():
            import pandas as pd
            metrics_df = pd.read_csv(metrics_path)
            self.thresholds = dict(zip(metrics_df['Label'], metrics_df['Threshold']))
        else:
            self.thresholds = {label: 0.5 for label in self.labels}
    
    def preprocess_image(self, image_path):
        """
        Preprocess the image for model input.
        
        Args:
            image_path: Path to the image file
            
        Returns:
            processed_image: Preprocessed image ready for model input
            original_image: Original image for visualization
        """
        # Read and resize image
        image = cv2.imread(image_path)
        original_image = image.copy()
        
        # Resize to model input size
        processed_image = cv2.resize(image, (224, 224))
        processed_image = processed_image.astype('float32') / 255.0
        
        return np.expand_dims(processed_image, axis=0), original_image
    
    def process_image(self, image_path):
        """
        Process the loaded image and save the result with highlighted constellations.
        
        Args:
            image_path: Path to the input image
        """
        # Preprocess image
        input_tensor, original_image = self.preprocess_image(image_path)
        
        # Get predictions
        predictions = self.model.predict(input_tensor)
        
        # Get detected constellations
        detected_constellations = []
        for i, label in enumerate(self.labels):
            if predictions[0][i] > self.thresholds[label]:
                confidence = predictions[0][i] * 100
                detected_constellations.append((label, confidence))
        
        # Highlight detected constellations
        highlighted_image = self.highlight_constellations(original_image, detected_constellations)
        
        # Save the output image
        output_path = os.path.splitext(image_path)[0] + "_output.jpg"
        cv2.imwrite(output_path, highlighted_image)
        print(f"Output saved to {output_path}")
    
    def highlight_constellations(self, image, detected_constellations):
        """
        Add visual highlights and labels for detected constellations.
        
        Args:
            image: Original image
            detected_constellations: List of (constellation_name, confidence) tuples
            
        Returns:
            highlighted_image: Image with highlights and labels
        """
        highlighted_image = image.copy()
        
        # Add text with detected constellations
        for i, (const, conf) in enumerate(detected_constellations):
            # Position text at the top of the image
            y_position = 30 + (i * 30)
            cv2.putText(
                highlighted_image,
                f"{const} ({conf:.1f}%)",
                (10, y_position),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 0),
                2
            )
        
        return highlighted_image

def main():
    # Set up paths
    model_path = r"D:\programs\astro_web\pretrained_model\output\constellation_model_densenet.keras"
    
    # File selection
    root = Tk()
    root.withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(
        title="Select an image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff")]
    )
    if not file_path:
        print("No file selected. Exiting.")
        return
    
    # Create and run the detector
    try:
        detector = ConstellationDetector(model_path)
        detector.process_image(file_path)
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Output saved to D:/programs/astro_web/pretrained_model/test/orion_constellation_043_png_jpg.rf.00830d963a648aa59f7964a6ddb65714_output.jpg
